In [1]:
#surrogate models
from autogluon.tabular import TabularDataset, TabularPredictor
from sklearn.model_selection import train_test_split

data_file = './all_parts_vectors_drags.csv'
df = TabularDataset(data_file)

train_df, test_df = train_test_split(df, test_size=0.2, random_state=777)

#exclue the first two columns of train data
train_data = train_df.drop(columns=['i', 'name'])
train_data.head()

/home/xli/anaconda3/envs/surrogate/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


,dim_1,dim_2,dim_3,dim_4,dim_5,dim_6,dim_7,dim_8,dim_9,dim_10,...,dim_440,dim_441,dim_442,dim_443,dim_444,dim_445,dim_446,dim_447,dim_448,drag
61,-6.639910,-2.575970,-4.333867,0.348634,-6.591602,4.137893,3.448946,9.801783,-4.944950,-1.340411,...,-1.080956,0.063173,1.612214,1.418663,0.173674,0.923131,-0.423641,-2.278858,-0.536435,0.375
354,5.044733,2.543681,-3.247805,-5.378260,0.549456,-1.075264,6.259791,5.443143,1.828774,5.774058,...,-1.829254,2.322891,0.782790,1.487128,-1.055578,2.655854,-0.162912,-0.385024,0.673573,0.374
358,-0.939149,0.704528,0.749587,-1.643254,-3.556550,0.293420,3.039790,-3.084175,-3.810488,3.759489,...,-0.636529,10.484594,10.753080,8.283512,1.325511,6.276447,-0.932776,11.217846,1.199296,0.435
275,-1.177986,1.175732,-1.834498,1.306601,-0.493694,1.298445,2.197183,4.541283,1.859666,3.284920,...,-2.321944,1.785326,0.453086,0.186109,1.353902,2.105992,-0.312803,-1.399713,0.909484,0.437
18,0.390329,-1.530953,-2.521950,-4.864147,-0.742480,0.146031,-0.156536,1.740131,-2.943921,5.280754,...,-2.205936,-0.899805,0.860134,1.484055,-2.426541,2.012407,-0.914859,-1.245621,0.841827,0.367


In [2]:
label = 'drag'
print("Summary of class variable: \n", train_data[label].describe())

Summary of class variable: 
 count    351.000000
mean       0.398513
std        0.060013
min        0.278000
25%        0.353000
50%        0.394000
75%        0.435000
max        0.598000
Name: drag, dtype: float64


In [6]:
import os
save_path = './agModels-all_parts'  # specifies folder to store trained models
if not os.path.exists(save_path):
    os.makedirs(save_path)

bag_folds = 5 #suggestion range [5, 10]
bag_sets = 3 #suggestion range [1, 20]
stack_levels = 3 #suggestion range [0, 3]
metric = 'root_mean_squared_error' #Regression:mean_absolute_error, mean_squared_error,root_mean_squared_error (default), r2
predictor = TabularPredictor(label=label, path=save_path, eval_metric=metric).fit(train_data, 
                                                                                  presets='best_quality', 
                                                                                  auto_stack="True", 
                                                                                  num_bag_folds=bag_folds, 
                                                                                  num_bag_sets=bag_sets,
                                                                                  num_stack_levels=stack_levels,
                                                                                  verbosity=4)

Presets specified: ['best_quality']
============ fit kwarg info ============
User Specified kwargs:
{'auto_stack': 'True',
 'num_bag_folds': 5,
 'num_bag_sets': 3,
 'num_stack_levels': 3,
 'verbosity': 4}
Full kwargs:
{'_feature_generator_kwargs': None,
 '_save_bag_folds': None,
 'ag_args': None,
 'ag_args_ensemble': None,
 'ag_args_fit': None,
 'auto_stack': 'True',
 'calibrate': 'auto',
 'excluded_model_types': None,
 'feature_generator': 'auto',
 'feature_prune_kwargs': None,
 'holdout_frac': None,
 'hyperparameter_tune_kwargs': None,
 'keep_only_best': False,
 'name_suffix': None,
 'num_bag_folds': 5,
 'num_bag_sets': 3,
 'num_stack_levels': 3,
 'pseudo_data': None,
 'refit_full': False,
 'save_space': False,
 'set_best_to_refit_full': False,
 'unlabeled_data': None,
 'use_bag_holdout': False,
 'verbosity': 4}
Stack configuration (auto_stack=True): num_stack_levels=3, num_bag_folds=5, num_bag_sets=3
Saving ./agModels-all_parts/learner.pkl
Saving ./agModels-all_parts/predictor.pkl
B

In [9]:
test_data = test_df.drop(columns=['i', 'name'])
# val_data.head()
y_val = test_data[label]
test_data_nolab = test_data.drop(columns=[label])  # delete label column to prove we're not cheating
test_data_nolab.head()

,dim_1,dim_2,dim_3,dim_4,dim_5,dim_6,dim_7,dim_8,dim_9,dim_10,...,dim_439,dim_440,dim_441,dim_442,dim_443,dim_444,dim_445,dim_446,dim_447,dim_448
46,3.262418,4.623484,5.207058,1.474604,5.149589,1.366774,7.350364,4.988407,-4.451728,2.715955,...,0.493204,-1.530669,-1.038711,1.546212,0.701456,-0.417296,2.205014,-2.338213,-1.252558,-0.207984
101,-1.815653,0.467063,-0.713573,-2.519432,-6.833046,3.925573,3.131740,2.785505,-0.169789,3.797550,...,-1.799442,-2.883593,1.992778,0.544198,1.757790,-0.963274,2.303173,-0.294457,-0.247084,2.795963
175,-0.128397,-0.347771,-4.491439,-0.143762,-1.139696,1.523660,4.283424,1.275272,-3.277825,-0.944104,...,-0.005584,-2.087350,0.265765,1.710699,-0.205781,0.221282,2.097013,-2.484610,-1.032043,0.472173
9,-0.334880,-0.358754,-5.121314,-1.869275,0.053525,2.998930,-0.482418,1.283854,-1.355309,-0.988547,...,2.702447,-1.539050,-0.802389,1.228584,1.375461,-2.406342,0.712337,-1.958422,-1.374814,-1.371111
136,0.339348,2.649365,-1.346519,0.872298,-2.672357,1.983254,2.075392,6.573480,-0.809723,-2.231186,...,4.364009,-0.522479,0.352676,0.769096,1.184941,2.701294,0.989609,-2.073087,-1.652956,-0.242727


In [10]:
%%capture log_output
%config InlineBackend.figure_format = 'retina'
%config Application.log_level = 'DEBUG'
%config IPCompleter.greedy = True

predictor = TabularPredictor.load(save_path)  # unnecessary, just demonstrates how to load previously-trained predictor from file
y_pred = predictor.predict(test_data_nolab)
for item in y_pred:
    print(item)
print("Predictions:  \n", y_pred)
perf = predictor.evaluate_predictions(y_true=y_val, y_pred=y_pred, auxiliary_metrics=True)
print(perf)

results = predictor.fit_summary(show_plot=True)
print(results)
print(predictor.leaderboard(test_data, silent=True))

with open('./output_all_parts.log', 'w') as f:
    f.write(log_output.stdout)

Loading: ./agModels-all_parts/predictor.pkl
Loading: ./agModels-all_parts/learner.pkl
Loading: ./agModels-all_parts/models/trainer.pkl
Loading: ./agModels-all_parts/models/CatBoost_BAG_L1/model.pkl
Loading: ./agModels-all_parts/models/ExtraTreesMSE_BAG_L1/model.pkl
Loading: ./agModels-all_parts/models/KNeighborsDist_BAG_L1/model.pkl
Loading: ./agModels-all_parts/models/KNeighborsUnif_BAG_L1/model.pkl
Loading: ./agModels-all_parts/models/LightGBMLarge_BAG_L1/model.pkl
Loading: ./agModels-all_parts/models/LightGBMXT_BAG_L1/model.pkl
Loading: ./agModels-all_parts/models/LightGBM_BAG_L1/model.pkl
Loading: ./agModels-all_parts/models/NeuralNetFastAI_BAG_L1/model.pkl
Loading: ./agModels-all_parts/models/NeuralNetTorch_BAG_L1/model.pkl
Loading: ./agModels-all_parts/models/RandomForestMSE_BAG_L1/model.pkl
Loading: ./agModels-all_parts/models/XGBoost_BAG_L1/model.pkl
Loading: ./agModels-all_parts/models/CatBoost_BAG_L2/model.pkl
Loading: ./agModels-all_parts/models/ExtraTreesMSE_BAG_L2/model.pk

In [11]:
print("AutoGluon infers problem type is: ", predictor.problem_type)
print("AutoGluon identified the following types of features:")
print(predictor.feature_metadata)

AutoGluon infers problem type is:  regression
AutoGluon identified the following types of features:
('float', []) : 448 | ['dim_1', 'dim_2', 'dim_3', 'dim_4', 'dim_5', ...]


In [12]:
train_data_pred = predictor.predict(train_data, model='WeightedEnsemble_L2')
for item in train_data_pred:
    print(item)

Loading: ./agModels-all_parts/models/LightGBMXT_BAG_L1/model.pkl
Loading: ./agModels-all_parts/models/LightGBM_BAG_L1/model.pkl
Loading: ./agModels-all_parts/models/NeuralNetFastAI_BAG_L1/model.pkl
Loading: ./agModels-all_parts/models/NeuralNetTorch_BAG_L1/model.pkl
Loading: ./agModels-all_parts/models/WeightedEnsemble_L2/model.pkl


0.39184391498565674
0.38747626543045044
0.4197539985179901
0.4179936647415161
0.3793541193008423
0.3751015067100525
0.44055819511413574
0.38958820700645447
0.3726438581943512
0.3715391159057617
0.3620454967021942
0.49061012268066406
0.44387468695640564
0.380361944437027
0.41504812240600586
0.4479149878025055
0.3274679183959961
0.4147564172744751
0.48429781198501587
0.3661191761493683
0.4156646430492401
0.45305365324020386
0.34406012296676636
0.42071548104286194
0.4072604477405548
0.41768452525138855
0.3798379600048065
0.41840359568595886
0.33708423376083374
0.35109180212020874
0.44828689098358154
0.41982996463775635
0.4782492518424988
0.43675580620765686
0.36623162031173706
0.3798624873161316
0.40959155559539795
0.4721307158470154
0.35714036226272583
0.3607688248157501
0.487091988325119
0.3831229507923126
0.35296615958213806
0.33161357045173645
0.3642938435077667
0.3847861886024475
0.3893302381038666
0.41547268629074097
0.35462716221809387
0.3950325548648834
0.3961527347564697
0.370400

In [13]:
test_data_pred = predictor.predict(test_data, model='WeightedEnsemble_L2')
for item in test_data_pred:
    print(item)

Loading: ./agModels-all_parts/models/LightGBMXT_BAG_L1/model.pkl
Loading: ./agModels-all_parts/models/LightGBM_BAG_L1/model.pkl
Loading: ./agModels-all_parts/models/NeuralNetFastAI_BAG_L1/model.pkl
Loading: ./agModels-all_parts/models/NeuralNetTorch_BAG_L1/model.pkl
Loading: ./agModels-all_parts/models/WeightedEnsemble_L2/model.pkl


0.384922057390213
0.38217824697494507
0.4016181230545044
0.4046119451522827
0.4317357540130615
0.4018993377685547
0.3992326259613037
0.44062340259552
0.4211746156215668
0.36477047204971313
0.4024960696697235
0.4058777391910553
0.37610968947410583
0.42764168977737427
0.40456944704055786
0.41403016448020935
0.3857561945915222
0.3971380591392517
0.3829162120819092
0.37643003463745117
0.4069160223007202
0.4054562747478485
0.37352097034454346
0.3897387683391571
0.4212469458580017
0.4098832607269287
0.4114890992641449
0.3821398913860321
0.4004173278808594
0.3805646300315857
0.4311654269695282
0.3803538680076599
0.40003323554992676
0.4070470333099365
0.389871209859848
0.3711824417114258
0.3777167499065399
0.41139066219329834
0.3946819603443146
0.38867270946502686
0.4345022439956665
0.3786471486091614
0.41710034012794495
0.3966688811779022
0.3841814696788788
0.3991056978702545
0.3795452415943146
0.36832818388938904
0.40859171748161316
0.394782155752182
0.4247400164604187
0.3549221456050873
0.4